In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession

In [2]:
spark = SparkSession.builder\
.appName("StructuredStreamingText2")\
.getOrCreate()

22/07/07 11:16:11 WARN Utils: Your hostname, ubuntu1-virtual-machine resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/07/07 11:16:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/07 11:16:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/07 11:16:12 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 71:f1:c3:85:14:23:37:db


In [3]:
from pyspark.sql.types import StructType

In [4]:
userSchema = StructType().add('name', 'string').add('salary', 'integer')

In [5]:
dfCSV = spark.readStream.option('sep', ';').option('header', 'false').schema(userSchema).csv('tmp')

In [10]:
import pyspark.sql.functions as F 
nameCountsDF = dfCSV.groupBy(dfCSV.name).agg(F.sum("salary"), F.count("*"))

In [11]:
query = nameCountsDF.writeStream.outputMode("complete").format("console").queryName("counts").start()

22/07/07 11:28:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-098e4643-fe8e-4144-a08f-b312a63b6507. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/07/07 11:28:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----------+--------+
| name|sum(salary)|count(1)|
+-----+-----------+--------+
|hello|          1|       1|
|world|          1|       1|
+-----+-----------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----------+--------+
| name|sum(salary)|count(1)|
+-----+-----------+--------+
|hello|        301|       2|
|world|          1|       1|
+-----+-----------+--------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----------+--------+
| name|sum(salary)|count(1)|
+-----+-----------+--------+
|hello|        301|       2|
|world|        101|       2|
+-----+-----------+--------+



In [ ]:
query.awaitTermination()

In [ ]:
query.stop()